In [5]:
import os
from typing import List, Optional, Dict
import xml.etree.ElementTree as ET
from tqdm import tqdm
import pandas as pd

def get_text(node):
    try:
        return node.text
    except AttributeError:
        return None


data_dir: str = '/media/wojtek/SanDisk/Medline/all'
file_extension: str = '.xml'

files: List[str] = os.listdir(data_dir)
files: List[str] = list(filter(lambda x: file_extension in x, files))
records_json = []
records_csv = []
with open(f'./out/pubmed.csv', 'w') as fw:
    for filename in tqdm(files):
        full_filename_path: str = os.path.join(data_dir, filename)
        tree = ET.parse(full_filename_path)
        root = tree.getroot()
        for mc in root.iter('MedlineCitation'):
            pmid_str: str = get_text(mc.find('PMID'))
            pmid: int = int(pmid_str) if pmid_str and len(pmid_str) > 0 else None
            year_str: str = get_text(mc.find('Article/Journal/JournalIssue/PubDate/Year'))
            year: int = int(year_str) if year_str and len(year_str) > 0 else None
            country: str = get_text(mc.find('MedlineJournalInfo/Country'))
            # authors: List[Dict] = []
            if year and year > 2016:
                for an in mc.findall('Article/AuthorList/Author'):
                    orcid = get_text(an.find('Identifier'))
                    affiliation = get_text(an.find('AffiliationInfo/Affiliation'))
                    # authors.append({
                    #     'surname': get_text(an.find('LastName')),
                    #     'name': get_text(an.find('ForeName')),
                    #     'orcid': orcid,
                    #     'affiliation': affiliation,
                    # })
                    if orcid is not None:
                        fw.write(f"{str(pmid)};{orcid};{affiliation};{year};{country}\n")
            # records_json.append({
            #     'pmid': pmid,
            #     'year': year,
            #     'authors': authors,
            # })
        # df = pd.DataFrame(records_csv)
        # df.to_csv(f"./out/{filename}.csv")

100%|██████████| 963/963 [4:32:29<00:00, 16.98s/it]  


In [23]:
import xml.etree.ElementTree as ET
tree = ET.parse('orcid.xml')
root = tree.getroot()
root.find('.//{http://www.orcid.org/ns/common}disambiguated-organization-identifier').text

'grid.1035.7'

In [32]:
'abc'.split('/')[-1]

'abc'

In [28]:
import xml.etree.ElementTree as ET
import requests
with open('./out/pubmed_no_aff.csv') as f:
    with open('./out/pubmed_from_orcid.csv', 'w') as fw:
        fw.write(f"pmid;orcid;aff_id_from_orcid;year;country_from_orcid\n")
        for line in tqdm(f, total=666713):
            try:
                pmid, orcid, year, affiliation_id, country_cleaned = line.split(';')
                response = requests.get(f'https://pub.orcid.org/v3.0/{orcid}/activities')
                root = ET.fromstring(response.content)
                org_id = root.find('.//{http://www.orcid.org/ns/common}disambiguated-organization-identifier').text
                country = root.find('.//{http://www.orcid.org/ns/common}country').text
                fw.write(f"{str(pmid)};{orcid};{org_id};{year};{country}\n")
            except:
                continue
    print(org_id, country)

grid.1035.7 PL
grid.1035.7 PL
grid.1035.7 PL
grid.1035.7 PL
grid.1035.7 PL
grid.1035.7 PL
grid.1035.7 PL
grid.1035.7 PL


KeyboardInterrupt: 

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<activities:activities-summary path="/0000-0002-1545-182X/activities" xmlns:internal="http://www.orcid.org/ns/internal" xmlns:education="http://www.orcid.org/ns/education" xmlns:distinction="http://www.orcid.org/ns/distinction" xmlns:deprecated="http://www.orcid.org/ns/deprecated" xmlns:other-name="http://www.orcid.org/ns/other-name" xmlns:membership="http://www.orcid.org/ns/membership" xmlns:error="http://www.orcid.org/ns/error" xmlns:common="http://www.orcid.org/ns/common" xmlns:record="http://www.orcid.org/ns/record" xmlns:personal-details="http://www.orcid.org/ns/personal-details" xmlns:keyword="http://www.orcid.org/ns/keyword" xmlns:email="http://www.orcid.org/ns/email" xmlns:external-identifier="http://www.orcid.org/ns/external-identifier" xmlns:funding="http://www.orcid.org/ns/funding" xmlns:preferences="http://www.orcid.org/ns/preferences" xmlns:address="http://www.orcid.org/ns/address" xmlns:invited-position="http://ww

In [15]:
root.getchildren()

<ipython-input-15-4340fffdc6c9>:1: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  root.getchildren()


[<Element '{http://www.orcid.org/ns/common}last-modified-date' at 0x7fd087486130>,
 <Element '{http://www.orcid.org/ns/activities}distinctions' at 0x7fd0874864a0>,
 <Element '{http://www.orcid.org/ns/activities}educations' at 0x7fd0874ae9f0>,
 <Element '{http://www.orcid.org/ns/activities}employments' at 0x7fd0874b7ea0>,
 <Element '{http://www.orcid.org/ns/activities}fundings' at 0x7fd0874ba3b0>,
 <Element '{http://www.orcid.org/ns/activities}invited-positions' at 0x7fd0874ba400>,
 <Element '{http://www.orcid.org/ns/activities}memberships' at 0x7fd0874ba4a0>,
 <Element '{http://www.orcid.org/ns/activities}peer-reviews' at 0x7fd0874ba540>,
 <Element '{http://www.orcid.org/ns/activities}qualifications' at 0x7fd0874ba590>,
 <Element '{http://www.orcid.org/ns/activities}research-resources' at 0x7fd0874ba5e0>,
 <Element '{http://www.orcid.org/ns/activities}services' at 0x7fd0874ba6d0>,
 <Element '{http://www.orcid.org/ns/activities}works' at 0x7fd0874ba7c0>]

In [37]:
from tqdm import tqdm

aff_to_num = dict()
current_index = 0
with open(f'./out/pubmed_no_aff.csv', 'w') as f_cleaned:
    with open(f'./out/pubmed.csv') as fw:
        f_cleaned.write(f"pmid;orcid;year;affiliation_id;country_cleaned\n")
        for line in tqdm(fw):
            try:
                pmid, orcid, affiliation, year, country = line.split(';')
            except ValueError:
                continue

            if 'USA' in affiliation:
                country_cleaned = 'USA'
            elif 'UK' in affiliation:
                country_cleaned = 'UK'
            elif 'Italy' in affiliation:
                country_cleaned = 'Italy'
            elif 'Poland' in affiliation:
                country_cleaned = 'Poland'
            else:
                continue
            if affiliation not in aff_to_num:
                aff_to_num[affiliation] = current_index
                current_index += 1
            f_cleaned.write(f"{pmid};{orcid};{year};{aff_to_num[affiliation]};{country_cleaned}\n")


2947134it [00:03, 978097.68it/s]


In [1]:
from requests import RequestException
import orcid

key = 'APP-6OIRWLFT0C1KZ9FD'
secret = 'dd06039c-9778-4ca8-adfc-f857be0c0fd7'

api = orcid.PublicAPI(key, secret, sandbox=True)
token = api.get_token('sitekwb@gmail.com', 'gerzaw-kUnde8-pogcef', 'https://pub.orcid.org')
api.read_record_public('0000-0001-8338-852X', 'ringgold-org-id')

AttributeError: 'NoneType' object has no attribute 'attrs'

In [ ]:
api.ge

In [3]:
token = api.get_token('sitekwb@gmail.com', 'gerzaw-kUnde8-pogcef', 'https://pub.orcid.org')

AttributeError: 'NoneType' object has no attribute 'attrs'

In [2]:
api.read_record_public('0000-0001-8338-852X', 'ringgold-org-id')

TypeError: read_record_public() missing 1 required positional argument: 'token'

In [35]:
current_index

397451

In [30]:
import pycountry
t = 'United Kingdom is a beautiful Australia USA'
for c in pycountry.countries:
    if c.name in t or c.alpha_2 in t:# or c.official_name in t:
        print(c.name)
        print(c.alpha_2)


Australia
AU
United Kingdom
GB
Saudi Arabia
SA
United States
US
